# Infer ***nuclei*** from a composite image - 2️⃣ 

--------------

## OVERVIEW
We will start by segmenting the different cell regions - the nucleus, cell, and cytoplasm - since they will be necessary for determining which organelle are in which cell. This is integral to our single cell analysis approach.

This notebook goes through the workflow steps to segment the ***nucleus*** from a fluorescent nuclei marker.


## OBJECTIVE: 
### ✅ Infer sub-cellular component #1: ***nuclei***
Segment the ***nuclei*** from a composite image of multiple organelle markers combined. The ***cell*** and ***cytoplasm*** masks will also be derived from the same composite image. 

> ***Biological relevance:***
> The combination of organelle markers used to create the composite image for the nucleus (and cell mask in [02_infer_cellmask_from-composite.ipynb](./02_infer_cellmask_from-composite.ipynb)) segmentation depends on the organelle labeles used and the cell type. In this example, the current selection includes the lysosomes, ER, and Golgi (e.g., Ch = 1, 3, 5) which have some intracellular background fluorescence - likely from off target marker localization. The lipid droplet channel (e.g., `ch = 6`) could also be included to produce a more unbiased selection of the entire cellmask as it binds to all cellular membranes to a small extent. However, the drawback of this is that it is present in every cell which makes downstream cell selection of a single cell more challenging. 
>
> *It is important to consider specifics of your system as the cell type and labeling method may differ from the example above.*

> ***Convention:***  "nuclei" for the segmentation of ALL nuclei in the image.  "nucleus" for the ***single*** nucleus associated to the single cell being analyzed.



### IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict
from typing import Optional

import numpy as np

from aicssegmentation.core.pre_processing_utils import  image_smoothing_gaussian_slice_by_slice 
from aicssegmentation.core.utils import hole_filling
from skimage.measure import label
import skimage


# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))


from infer_subc.utils.file_io import (read_czi_image,
                                                                    list_image_files)
from infer_subc.utils.img import *
from infer_subc.organelles import fixed_get_optimal_Z_image, fixed_find_optimal_Z, find_optimal_Z
from infer_subc.constants import (TEST_IMG_N,
                                                                    NUC_CH ,
                                                                    LYSO_CH ,
                                                                    MITO_CH ,
                                                                    GOLGI_CH ,
                                                                    PEROXI_CH ,
                                                                    ER_CH ,
                                                                    LIPID_CH ,
                                                                    RESIDUAL_CH, 
                                                                    ALL_CHANNELS )          

from infer_subc.organelles import infer_soma, fixed_infer_soma

%load_ext autoreload
%autoreload 2

## Get and load Image for processing

In [ ]:
test_img_n = TEST_IMG_N

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python Scripts/Infer-subc-2D"

in_data_path = data_root_path / "raw"
im_type = ".czi"

img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

out_data_path = data_root_path / "out"
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [ ]:
img_data,meta_dict = read_czi_image(test_img_name)

channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\ome_types\_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


---------
## infer ***nuclei*** from composite image

### summary of steps

➡️ INPUT
- create composite image from multiple organelle channels

PRE-PROCESSING
- scale to min 0, max 1.0
- ~~median Filter window 4~~
- ~~gaussian 1.34~~

CORE-PROCESSING
  - threshold method otsu  
    - threshold correction factor: 0.05
    - lower / upper bounds  (0, 1)
  - invert segmentation

POST-PROCESSING
  - fill holes
  - remove small objects
  - label
  - remove objects touching image edges

OUTPUT ➡️ 
- mask of NUCLEI


## INPUT (prototype)

Get the "raw" signals we need to analyze as well as any other dependencies in "inferred" objects.  

In [4]:
# ###################
# # INPUT
# ###################
# # raw_nuclei = img_data[NUC_CH].copy()
# raw_nuclei = select_channel_from_raw(img_data, NUC_CH)

# print(raw_nuclei.shape)

raw_nuclei = img_data[1]
print(raw_nuclei.shape)

(49, 1688, 1688)


## PRE-PROCESSING (prototype)

In [5]:
###################
# PRE_PROCESSING
###################           
nuclei_norm = min_max_intensity_normalization(raw_nuclei)

#### smoothing is no longer necessary here - Huygens preprocessing took care of that already

# med_filter_size = 4   
# nuclei_med = median_filter_slice_by_slice(nuclei_norm,
#                                       size=med_filter_size)
# print(nuclei_med.shape)

# gaussian_smoothing_sigma = 1.34
# nuclei_gaus = image_smoothing_gaussian_slice_by_slice(nuclei_med,
#                                                       sigma=gaussian_smoothing_sigma)
# print(nuclei_gaus.shape)


### VISUALIZE: the nuclei image after pre-processing
Use this to adjust median filter size and gaussian sigma above.

In [6]:
viewer = napari.Viewer()

15-Mar-23 16:01:10 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1090x686+1+15 (frame: 1108x733-8-23) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 1365x859+2+22 (frame: 1383x906-7-16) margins: 9, 38, 9, 9 minimum size: 612x589 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=630,636 maxtrack=0,0)


In [7]:
viewer.add_image(
    raw_nuclei,
    scale=scale
)
viewer.add_image(
    nuclei_norm,
    scale=scale
)
# viewer.add_image(
#     nuclei_gaus,
#     scale=scale
# )

<Image layer 'nuclei_norm' at 0x22fcfeab640>

## CORE PROCESSING (prototype)


> #### ASIDE: Thresholding
> [Thresholding](https://en.wikipedia.org/wiki/Thresholding_%28image_processing%29) is used to create binary images. A threshold value determines the intensity value separating foreground pixels from background pixels. Foregound pixels are pixels brighter than the threshold value, background pixels are darker. In many cases, images can be adequately segmented by thresholding followed by labelling of *connected components*, which is a fancy way of saying "groups of pixels that touch each other".
> 
> Different thresholding algorithms produce different results. [Otsu's method](https://en.wikipedia.org/wiki/Otsu%27s_method) and [Li's minimum cross entropy threshold](https://scikit-image.org/docs/dev/auto_examples/developers/plot_threshold_li.html) are two common algorithms. Below, we use Li. You can use `skimage.filters.threshold_<TAB>` to find different thresholding methods.


In [8]:

###################
# CORE_PROCESSING
###################
threshold_factor = 0.05 #from cellProfiler
thresh_min = 0
thresh_max = 1
threshold = apply_threshold(nuclei_norm, thresh_factor=threshold_factor, thresh_min=thresh_min, thresh_max=thresh_max)
bw = nuclei_norm < threshold
bw_invert = 1 - bw

bw_labeled = label(bw_invert, connectivity=1)


### VISUALIZE: the nuclei image after core processing
Use this to adjust threshold parameters above.

In [9]:
viewer.add_image(
    bw,
    scale=scale
)
viewer.add_image(
    bw_invert,
    scale=scale
)
viewer.add_labels(
    bw_labeled,
    scale=scale
)

<Labels layer 'bw_labeled' at 0x22f80708a30>

## POST PROCESSING prototype

> NOTE: the size parameters are by convention defined as one dimensional "width", so the inputs to the functions need to be _squared_ i.e. raised to the power of 2: `** 2`.   For volumetric (3D) analysis this would be _cubed_:`**3`

In [10]:
###################
# POST_PROCESSING
###################

hole_width = 30 
removed_holes = hole_filling(bw_invert,
                             hole_min=0, 
                             hole_max=hole_width**3, 
                             fill_2d=False)

small_object_width = 20
cleaned_img = size_filter(removed_holes, 
                          min_size= small_object_width**3, #changed this to 3 to adjust for the 3D voxel, instead of a 2D pixel 
                          method="3D",
                          connectivity=1)

all_labels = label(cleaned_img, connectivity=1)

nuclei_label = skimage.segmentation.clear_border(all_labels)


### VISUALIZE: the nuclei image after post-processing
Use this to adjust the size parameters above.

In [11]:
viewer.add_image(
    removed_holes,
    scale=scale
)
viewer.add_image(
    cleaned_img,
    scale=scale
)
viewer.add_labels(
    all_labels,
    scale=scale
)
viewer.add_labels(
    nuclei_label,
    scale=scale
)

<Labels layer 'nuclei_label' at 0x22f80a79eb0>

In [12]:
viewer.close()

### Adjust naming and type for labels

In [23]:
NU_signal = raw_nuclei

# renaming semantic segmentation of nuclei
nuclei_object = cleaned_img

# creating instance segmentations for all nuclei in nuclei_object
NU_labels = label(cleaned_img)

### VISUALIZE final segmentations

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.add_image(
    NU_signal,
    scale=scale,
)
viewer.add_image(
    nuclei_object,
    scale=scale,
    opacity=0.3,
)    
viewer.add_labels(
    NU_labels,
    scale=scale,
    opacity=0.3,
)

In [ ]:
viewer.close()

## DEFINE `_infer_nuclei` function

Based on the _prototyping_ above define the function to infer nuclei.  


In [16]:
##########################
#  _infer_nuclei
##########################
def _infer_nucleifromER_3D( in_img: np.ndarray,
                       thresh_factor: float,
                       thresh_min: float,
                       thresh_max: float,
                       max_hole_w: int,
                       small_obj_w: int,
                       sz_filter_method: str
                     ) -> np.ndarray:
    """
    Procedure to infer 3D nuclei segmentation from multichannel z-stack input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels
    cellmask: Optional[np.ndarray] = None
        mask
    thresh_factor: float
        adjustment factor for log Li threholding
    thresh_min: float
        abs min threhold for log Li threholding
    thresh_max: float
        abs max threhold for log Li threholding
    max_hole_w: int
        hole filling cutoff for nuclei post-processing
    small_obj_w: int
        minimum object size cutoff for nuclei post-processing
    sz_filter_method: str
        method for size filtering; either "3D" or "slice_by_slice"

    Returns
    -------------
    nuclei_object
        mask defined extent of NU
    
    """

    # nuc_ch_from_ER = ER_CH
    # nuclei = select_channel_from_raw(in_img, nuc_ch_from_ER)
    nuclei = in_img[1]


    ###################
    # PRE_PROCESSING
    ###################                
    nuclei = min_max_intensity_normalization(nuclei)


    ###################
    # CORE_PROCESSING
    ###################
    thresh_value = apply_threshold(nuclei, 
                                   thresh_factor=thresh_factor, 
                                   thresh_min=thresh_min, 
                                   thresh_max=thresh_max)
    nuclei_object = nuclei < thresh_value
    nuclei_object = 1 - nuclei_object


    ###################
    # POST_PROCESSING
    ###################
    nuclei_object = hole_filling(nuclei_object, 
                                 hole_min=0, 
                                 hole_max=max_hole_w**3, 
                                 fill_2d=False)

    nuclei_object = size_filter(nuclei_object, 
                                min_size = small_obj_w**3, 
                                method = sz_filter_method,
                                connectivity=1)
    
    nuclei_object = label(nuclei_object, connectivity=1)
    
    nuclei_object = skimage.segmentation.clear_border(nuclei_object)

    return nuclei_object


## DEFINE `_fixed_infer_nuclei` function


In [19]:
##########################
#  fixed_infer_nuclei
##########################
def _fixed_infer_nuclei_3D(in_img: np.ndarray) -> np.ndarray:
    """
    Procedure to infer soma from linearly unmixed input, with a *fixed* set of parameters for each step in the procedure.  i.e. "hard coded"

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels
    cellmask: np.ndarray
        mask
 
    Returns
    -------------
    nuclei_object
        mask defined extent of NU
    
    """

    nuc_ch = NUC_CH
    threshold_factor = 0.05
    thresh_min = 0
    thresh_max = 1.0
    max_hole_w = 30
    small_obj_w = 20
    sz_filter_method = "3D"

    return _infer_nucleifromER_3D( in_img,
                             threshold_factor,
                             thresh_min,
                             thresh_max,
                             max_hole_w,
                             small_obj_w,
                             sz_filter_method )


---------------------
## TEST `_infer_nuclei`  function defined above


In [20]:
_NU_object =  _fixed_infer_nuclei_3D(img_data) 

In [21]:
viewer = napari.Viewer()

15-Mar-23 16:08:23 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1090x686+2+15 (frame: 1108x733-7-23) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 1365x859+3+22 (frame: 1383x906-6-16) margins: 9, 38, 9, 9 minimum size: 612x589 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=630,636 maxtrack=0,0)


In [24]:
viewer.add_image(
    NU_signal,
    scale=scale,
)
viewer.add_image(
    nuclei_object,
    scale=scale,
    opacity=0.3,
)    
viewer.add_labels(
    NU_labels,
    scale=scale,
    opacity=0.3,
)
viewer.add_image(
    _NU_object,
    scale=scale
)

<Image layer '_NU_object' at 0x22fac2096a0>

In [ ]:
viewer.close()

<code style="background:yellow;color:black"> ***SR edits stops here***</code>

---------------------
# TEST `infer_nuclei` exported functions

> the prototype `_infer_nuclei` was copied to the [`.organelles.nuclei`](../infer_subc/organelles/nuclei.py) sub-module 
##
`infer_nuclei` procedure

Use the `infer_nuclei` function to infer the Nucleus and export it as an _ome.tif_ for easy reference.

In [ ]:
from infer_subc.organelles.nuclei import infer_nuclei, fixed_infer_nuclei

nuclei_object =  fixed_infer_nuclei(img_2D, cellmask) 


## Visualize  2


In [ ]:
viewer.add_image(
    _NU_object,
    scale=scale,
    opacity=0.3,
)    


viewer.add_labels(
    label(_NU_object),
    scale=scale,
    opacity=0.3,
)


In [ ]:
viewer.add_image(
    nuclei_object,
    scale=scale,
    opacity=0.3,
)    


In [ ]:
from napari.utils.notebook_display import nbscreenshot

# viewer.dims.ndisplay = 3
# viewer.camera.angles = (-30, 25, 120)
nbscreenshot(viewer, canvas_only=True)


In [ ]:
viewer.close()

-----------------
Write the `infer_nuclei` spec to the widget json 

In [ ]:
from infer_subc.organelles_config.helper import add_function_spec_to_widget_json

_fixed_infer_nuclei =  {
        "name": "infer nuclei  (fixed parameters)",
        "python::module": "infer_subc.organelles",
        "python::function": "fixed_infer_nuclei",
        "parameters": None
        }

add_function_spec_to_widget_json("fixed_infer_nuclei",_fixed_infer_nuclei)

In [ ]:

_infer_nuclei =  {
        "name": "infer nuclei",
        "python::module": "infer_subc.organelles",
        "python::function": "infer_nuclei",
        "parameters": {
                "median_sz": {
                        "widget_type": "slider",
                        "data_type": "int",
                        "min": 3,
                        "max": 15,
                        "increment": 1
                },
                "gauss_sig": {
                        "data_type": "float",
                        "increment": 0.25,
                        "max": 15.0,
                        "min": 1.25,
                        "widget_type": "slider"
                },
                "thresh_factor": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": 1.2,
                        "min": 0.6,
                        "widget_type": "slider"
                },
                "thresh_min": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": .9,
                        "min": 0.0,
                        "widget_type": "slider"
                },
                "thresh_max": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": 1.0,
                        "min": 0.1,
                        "widget_type": "slider"
                },
                "max_hole_w": {
                        "data_type": "int",
                        "increment": 1,
                        "max": 40,
                        "min": 4,
                        "widget_type": "slider"
                },           
                "small_obj_w": {
                        "data_type": "int",
                        "increment": 1,
                        "max": 50,
                        "min": 1,
                        "widget_type": "slider"
                }
        }
}

add_function_spec_to_widget_json("infer_nuclei", _infer_nuclei, overwrite=True )



In [ ]:
_median_filter_slice_by_slice =  {
                "name": "Median Smoothing Slice by Slice",
                "python::module": "infer_subc.utils.img",
                "python::function": "median_filter_slice_by_slice",
                "parameters": {
                    "size": {
                        "widget_type": "slider",
                        "data_type": "int",
                        "min": 1,
                        "max": 20,
                        "increment": 1
                    }
                }
            } 
add_function_spec_to_widget_json("median_filter_slice_by_slice",_median_filter_slice_by_slice)

In [ ]:


_image_smoothing_gaussian_slice_by_slice = {
        "name": "Gaussian Smoothing Slice by Slice",
        "python::module": "aicssegmentation.core.pre_processing_utils",
        "python::function": "image_smoothing_gaussian_slice_by_slice",
        "parameters": {
            "sigma": {
                "widget_type": "slider",
                "data_type": "float",
                "min": 0.8,
                "max": 20,
                "increment": 0.2
            }
        }
        }

# json.dumps({"image_smoothing_gaussian_slice_by_slice": _image_smoothing_gaussian_slice_by_slice} )
add_function_spec_to_widget_json("image_smoothing_gaussian_slice_by_slice",_image_smoothing_gaussian_slice_by_slice)        


In [ ]:

# WARNING: not a good way to set to None
_apply_log_li_threshold = {
        "name": "threshold log Li",
        "python::module": "infer_subc.utils.img",
        "python::function": "apply_log_li_threshold",
        "parameters": {
            "thresh_factor": {
                "widget_type": "slider",
                "data_type": "float",
                "min": 0.3,
                "max": 1.1,
                "increment": 0.05
            },
            "thresh_min": {
                "widget_type": "slider",
                "data_type": "float",
                "min": 0.0,
                "max": 0.8,
                "increment": 0.01
            },
            "thresh_max": {
                "widget_type": "slider",
                "data_type": "float",
                "min": 0.3,
                "max": 1.0,
                "increment": 0.05
            },
        }
        }

# json.dumps({"apply_log_li_threshold": _apply_log_li_threshold} )
add_function_spec_to_widget_json("apply_log_li_threshold",_apply_log_li_threshold,overwrite=True)        


In [ ]:


    # NU_labels = label(nuclei_object)

_label =  {
        "name": "label objects",
        "python::module": "skimage.measure",
        "python::function": "label",
        "parameters": None
        }
# json.dumps({"label":_label})
add_function_spec_to_widget_json("label",_label)        


In [ ]:

#  nulei_object = apply_mask(nuclei_object, cellmask)

_apply_mask=  {
        "name": "apply mask",
        "python::module": "infer_subc.utils.img",
        "python::function": "apply_mask",
        "parameters": None
        }
# json.dumps({"apply_mask":_apply_mask})
add_function_spec_to_widget_json("apply_mask",_apply_mask)        


In [ ]:

    # small_object_width = 45
    # nuclei_object = size_filter_2D(nuclei_object, 
    #                                                             min_size= small_object_width**2, 
    #                                                             connectivity=1)


_size_filter_2D = {
        "name": "Size Filter 2D",
        "python::module": "infer_subc.utils.img",
        "python::function": "size_filter_2D",
        "parameters": {
            "min_size": {
                "widget_type": "slider",
                "data_type": "int",
                "min": 0,
                "max": 500,
                "increment": 1
            }
        }
    }
# json.dumps({  "size_filter_2D":  _size_filter_2D   })

add_function_spec_to_widget_json("size_filter_2D",_size_filter_2D)        


-------------------------------
## Write workflow .json
Now that we've added our function specs we can compose workflows.

In [ ]:
from infer_subc.constants import NUC_CH


def make_infer_nuclei_dict():
    """
    Procedure to infer nuclei from linearly unmixed input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels

    cellmask: np.ndarray
        mask

    Returns
    -------------
    nuclei_object
        mask defined extent of NU

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   
    step_name.append("1")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = NUC_CH) )
    parent.append(0)

    ###################
    # PRE_PROCESSING
    ###################                         
    # nuclei = min_max_intensity_normalization(in_img[NUC_CH].copy() )
    step_name.append("2")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(1)

    # size = 4   
    # nuclei = median_filter_slice_by_slice( 
    #                                                                 nuclei,
    #                                                                 size=size  )
    step_name.append("3")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 4 ))
    parent.append(2)

    # sigma = 1.34
    # truncate_range = 3.0
    # nuclei = image_smoothing_gaussian_slice_by_slice(  nuclei,
    #                                                                                             sigma=sigma,
    #                                                                                             truncate_range = truncate_range
    #                                                                                             )
    step_name.append("4")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.34 ))
    parent.append(3)

    ###################
    # CORE_PROCESSING
    ###################
    # threshold_factor = 0.9 
    # thresh_min = .1
    # thresh_max = 1.
    # nuclei_object = apply_log_li_threshold(nuclei, threshold_factor=threshold_factor, thresh_min=thresh_min, thresh_max=thresh_max)
    step_name.append("5")
    function_name.append("apply_log_li_threshold")
    category.append("core")
    parameter_values.append(dict(thresh_factor = 0.9, 
                                                            thresh_min = .1,
                                                            thresh_max = 1.))
    parent.append(4)


    # NU_labels = label(nuclei_object)
    step_name.append("6")
    function_name.append("label")
    category.append("core")
    parameter_values.append(None)
    parent.append(5)


    ###################
    # POST_PROCESSING
    ###################
    # hole_width = 5  
    # nuclei_object = hole_filling(nuclei_object, hole_min=0, hole_max=hole_width**2, fill_2d=True)
    step_name.append("6")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=5**2, fill_2d=True))
    parent.append(5)

    # # EEEEEK I don't know how to compose where the mask comes from... 
    # nuclei_object = apply_mask(nuclei_object, cellmask)

    # small_object_width = 15
    # nuclei_object = size_filter_2D(nuclei_object, 
    #                                                             min_size= small_object_width**2, 
    #                                                             connectivity=1)
    step_name.append("7")
    function_name.append("size_filter_2D")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 15**2  ))
    parent.append(6)

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:
from infer_subc.utils.directories import Directories
import json

def _write_workflow_json(wf_name, wf_dict):

    # read all_functions.json into dict
    # if not wf_name.startswith("conf"):
    #     wf_name = f"conf_{wf_name}"
    path = Directories.get_structure_config_dir() / f"{wf_name}.json"

    # re-write file
    with open(path, "w") as file:
        json.dump(wf_dict, file, indent=4, sort_keys=False)

    return path




In [ ]:
from infer_subc.organelles_config.helper import write_workflow_json

infer_nuclei_dict = make_infer_nuclei_dict()

write_workflow_json("conf_2.2.nuclei_stepbystep", infer_nuclei_dict)

In [ ]:
from infer_subc.constants import NUC_CH


def make_infer_nuclei_from_raw_dict():
    """
    Procedure to infer nuclei from linearly unmixed input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels

    cellmask: np.ndarray
        mask

    Returns
    -------------
    nuclei_object
        mask defined extent of NU

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   

    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)


    step_name.append("2")
    function_name.append("fixed_infer_soma")
    category.append("extraction")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = NUC_CH) )
    parent.append(1)

    ###################
    # PRE_PROCESSING
    ###################                         
    # nuclei = min_max_intensity_normalization(in_img[NUC_CH].copy() )
    step_name.append("4")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(3)

    # size = 4   
    # nuclei = median_filter_slice_by_slice( 
    #                                                                 nuclei,
    #                                                                 size=size  )
    step_name.append("5")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 4 ))
    parent.append(4)

    # sigma = 1.34
    # truncate_range = 3.0
    # nuclei = image_smoothing_gaussian_slice_by_slice(  nuclei,
    #                                                                                             sigma=sigma,
    #                                                                                             truncate_range = truncate_range
    #                                                                                             )
    step_name.append("6")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.34 ))
    parent.append(5)

    ###################
    # CORE_PROCESSING
    ###################
    # threshold_factor = 0.9 
    # thresh_min = .1
    # thresh_max = 1.
    # nuclei_object = apply_log_li_threshold(nuclei, threshold_factor=threshold_factor, thresh_min=thresh_min, thresh_max=thresh_max)
    step_name.append("7")
    function_name.append("apply_log_li_threshold")
    category.append("core")
    parameter_values.append(dict(thresh_factor = 0.9, 
                                                            thresh_min = .1,
                                                            thresh_max = 1.))
    parent.append(6)


    # NU_labels = label(nuclei_object)
    step_name.append("8")
    function_name.append("label")
    category.append("core")
    parameter_values.append(None)
    parent.append(7)


    ###################
    # POST_PROCESSING
    ###################
    # hole_width = 5  
    # nuclei_object = hole_filling(nuclei_object, hole_min=0, hole_max=hole_width**2, fill_2d=True)
    step_name.append("9")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=5**2, fill_2d=True))
    parent.append(8)

    step_name.append("10")
    function_name.append("apply_mask")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([9,2])


    # small_object_width = 15
    # nuclei_object = size_filter_2D(nuclei_object, 
    #                                                             min_size= small_object_width**2, 
    #                                                             connectivity=1)
    step_name.append("11")
    function_name.append("size_filter_2D")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 15**2  ))
    parent.append(10)

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:
from infer_subc.organelles_config.helper import write_workflow_json

infer_nuclei_dict = make_infer_nuclei_from_raw_dict()

write_workflow_json("conf_1.2.nuclei_stepbystep_from_raw", infer_nuclei_dict)

In [ ]:
from infer_subc.organelles_config.helper import add_function_spec_to_widget_json

_infer_nuclei =  {
        "name": "infer infer_nuclei",
        "python::module": "infer_subc.organelles",
        "python::function": "infer_nuclei",
        "parameters": None
        }

add_function_spec_to_widget_json("infer_nuclei",_infer_nuclei)

-------------
## SUMMARY

The above details how the nuclei object is inferred.  

### NEXT: INFER CYTOSOL

proceed to [03_infer_cytosol.ipynb](./03_infer_cytosol.ipynb)
